In [35]:
import pyreadstat
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [36]:
don=pd.read_csv("../1_data/private/donation_donor.csv") #donations

In [37]:
#preprocess don

don.loc[(don['PHLEBOTOMY_GROUP'] == 'Whole Blood'), 'don_proc_cat']='WB'

don.loc[(don['PHLEBOTOMY_GROUP'] == 'RBCP') |
          (don['PHLEBOTOMY_GROUP'] == 'Platelet/RBC/Plasma') | (don['PHLEBOTOMY_GROUP'] == 'Platelet/RBC'), 'don_proc_cat']='RBC'

don.loc[(don['PHLEBOTOMY_GROUP'] == 'Platelet') | (don['PHLEBOTOMY_GROUP'] == 'Platelet/Plasma') |
          (don['PHLEBOTOMY_GROUP'] == 'Plasma') | (don['PHLEBOTOMY_GROUP'] == 'COVID-19 Plasma') | (don['PHLEBOTOMY_GROUP'] == 'Source Plasma'), 'don_proc_cat']='Platelet/Plasma'

don.loc[(don['PHLEBOTOMY_GROUP'] == 'Sample Only') | (don['PHLEBOTOMY_GROUP'] == 'Research/Sample Only'), 'don_proc_cat']='Sample'

don.loc[(don['PHLEBOTOMY_GROUP'] == 'Not Drawn') | (don['PHLEBOTOMY_GROUP'] == 'UNKNOWN'), 'don_proc_cat']='Not Drawn/Unknown'

don.loc[(don['PHLEBOTOMY_GROUP'] == '2RBC') , 'don_proc_cat']='2RBC'

don.loc[(don['PHLEBOTOMY_GROUP'] == 'Tissue Collections') | (don['PHLEBOTOMY_GROUP'] == 'Granulocyte'), 'don_proc_cat']='Tissue/Granulocyte'

don.don_proc_cat.value_counts()


WB                    6191008
Platelet/Plasma       1109299
Not Drawn/Unknown      829207
2RBC                   804348
RBC                    145233
Sample                  14514
Tissue/Granulocyte       1584
Name: don_proc_cat, dtype: int64

In [38]:
don.sort_values(by=['DONOR_KEY', 'DON_DATE_KEY', 'DIN'], axis = 0, inplace=True)
don["NEXT_DON"] = np.where(don['DONOR_KEY'] == don['DONOR_KEY'].shift(-1), don['don_proc_cat'].shift(-1), 10)
don["PREV_DON"] = np.where(don['DONOR_KEY'] == don['DONOR_KEY'].shift(1), don['don_proc_cat'].shift(1), 10)

In [39]:
#IMPUTE DON_PROC
# if immediate next donation procedure is WB/RBC then deferred donation is WB/RBC
 #if immediate previous donation procedure is WB/RBC then deferred donation is WB/RBC
# if only donated one time and got deferred - WB/RBC


don['def_prod_imputed'] = np.where((don['don_proc_cat'] == 'Not Drawn/Unknown') & (don.PREV_DON=='WB'), 1, 0)
don['def_prod_imputed'] = np.where((don['don_proc_cat'] == 'Not Drawn/Unknown') & (don.NEXT_DON=='WB'), 1, don.def_prod_imputed)
don['def_prod_imputed'] = np.where((don['don_proc_cat'] == 'Not Drawn/Unknown') & ((don.NEXT_DON==10) & (don.PREV_DON==10)), 1, don.def_prod_imputed)


In [40]:
don['don_proc_cat']=np.where(don.def_prod_imputed == 1, 'WB', don.don_proc_cat)
don.don_proc_cat.value_counts()

WB                    6836654
Platelet/Plasma       1109299
2RBC                   804348
Not Drawn/Unknown      183561
RBC                    145233
Sample                  14514
Tissue/Granulocyte       1584
Name: don_proc_cat, dtype: int64

In [41]:
# backward fill remaining NOT DRAWNs- most proximal next visit

don['don_proc_cat']=np.where((don['don_proc_cat'] == 'Not Drawn/Unknown'), np.NaN, don.don_proc_cat)

don['don_proc_cat'] = don.groupby('DONOR_KEY')['don_proc_cat'].bfill()
don.don_proc_cat.value_counts()

WB                    6859594
Platelet/Plasma       1159562
2RBC                   823730
RBC                    151344
Sample                  15208
Tissue/Granulocyte       1723
Name: don_proc_cat, dtype: int64

In [42]:
#don=don2#uncomment and run to reset
don2=don #duplicate before changing
don.drop(don[(don['DONOR_NUMBER'] == 'UNKNOWN')].index, inplace=True)
don.drop(don[(don['DONOR_KEY'] == 0.0)].index, inplace=True)
don.drop(don[don.DON_DATE_KEY.isna()].index, inplace=True)

#rbc and Wb only and allogenic
don.drop(don[(don['don_proc_cat'] != 'WB')].index, inplace=True)
don.drop(don[(don['DONATION_TYPE'] != 'Allogeneic')].index, inplace=True)
#remove FINs that are still in the process if migrating

don['FIN']=don.DONATION_NUMBER.str.slice(start=0, stop=5)
don.drop(don[(don['FIN'] == 'W0358') |
             (don['FIN'] == 'W0364') |
             (don['FIN'] == 'W0433') |
             (don['FIN'] == 'W2864') |
             (don['FIN'] == 'W2865') |
             (don['FIN'] == 'W2868') |
             (don['FIN'] == 'W2869') |
             (don['FIN'] == 'W2870') |
             (don['FIN'] == 'W2873') |
             (don['FIN'] == 'W2874') |
             (don['FIN'] == 'W2875') |
             (don['FIN'] == 'W2876') |
             (don['FIN'] == 'W2877') |
             (don['FIN'] == 'W2878') |
             (don['FIN'] == 'W2879') |
             (don['FIN'] == 'W2880') |
             (don['FIN'] == 'W2881') |
             (don['FIN'] == 'W2882') |
             (don['FIN'] == 'W2883') |
             (don['FIN'] == 'W2884') |
             (don['FIN'] == 'W2885') |
             (don['FIN'] == 'W2886') ].index, inplace=True)

len(don)

5715779

In [44]:
#save preprocessed file
don.to_csv(("../1_data/private/donation_donor2.csv"), index=False)